In [45]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import sklearn

In [2]:
# 导入数据
breast_cancer = datasets.load_breast_cancer()

In [3]:
# 观察数据结构
print(breast_cancer.keys())
print(breast_cancer.data.shape,breast_cancer.target.shape)
print(breast_cancer.data[0])

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])
(569, 30) (569,)
[  1.79900000e+01   1.03800000e+01   1.22800000e+02   1.00100000e+03
   1.18400000e-01   2.77600000e-01   3.00100000e-01   1.47100000e-01
   2.41900000e-01   7.87100000e-02   1.09500000e+00   9.05300000e-01
   8.58900000e+00   1.53400000e+02   6.39900000e-03   4.90400000e-02
   5.37300000e-02   1.58700000e-02   3.00300000e-02   6.19300000e-03
   2.53800000e+01   1.73300000e+01   1.84600000e+02   2.01900000e+03
   1.62200000e-01   6.65600000e-01   7.11900000e-01   2.65400000e-01
   4.60100000e-01   1.18900000e-01]


### 数据标准化

In [9]:
std_x = StandardScaler()
data = std_x.fit_transform(breast_cancer.data)
print(data[0])
# 把数据分为数据集和测试集
std_X_train,std_X_test,std_y_train,std_y_test = train_test_split(data,breast_cancer.target,test_size=0.25,random_state=0)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

[ 1.09706398 -2.07333501  1.26993369  0.9843749   1.56846633  3.28351467
  2.65287398  2.53247522  2.21751501  2.25574689  2.48973393 -0.56526506
  2.83303087  2.48757756 -0.21400165  1.31686157  0.72402616  0.66081994
  1.14875667  0.90708308  1.88668963 -1.35929347  2.30360062  2.00123749
  1.30768627  2.61666502  2.10952635  2.29607613  2.75062224  1.93701461]
(426, 30) (426,)
(143, 30) (143,)


### knn模型---数据标准化之后

In [10]:
# 模型实例化并加载数据
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(std_X_train,std_y_train)
print("训练集精度：",knn.score(std_X_train,std_y_train))
print("测试集精度：",knn.score(std_X_test,std_y_test))

训练集精度： 0.983568075117
测试集精度： 0.944055944056


### knn模型---数据标准化之前

In [6]:
X_train,X_test,y_train,y_test = train_test_split(breast_cancer.data,breast_cancer.target,test_size=0.25,random_state=0)
# 模型实例化并加载数据
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
print("训练集精度：",knn.score(X_train,y_train))
print("测试集精度：",knn.score(X_test,y_test))

训练集精度： 0.957746478873
测试集精度： 0.923076923077


### 决策树

In [44]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=5)
tree.fit(std_X_train,std_y_train)
print("训练集精度：",tree.score(std_X_train,std_y_train))
print("测试集精度：",tree.score(std_X_test,std_y_test))
print(tree)

训练集精度： 0.992957746479
测试集精度： 0.93006993007
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


### 随机森林

In [69]:
from sklearn.ensemble import RandomForestClassifier
rand_forest = RandomForestClassifier(max_depth=8,min_samples_leaf=7)
rand_forest.fit(std_X_train,std_y_train)
print("训练集精度：",rand_forest.score(std_X_train,std_y_train))
print("测试集精度：",rand_forest.score(std_X_test,std_y_test))
print(rand_forest)

训练集精度： 0.967136150235
测试集精度： 0.965034965035
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


### sklearn中的人工神经网络库

In [165]:
# 标准化之前
X_train,X_test,y_train,y_test = train_test_split(breast_cancer.data,breast_cancer.target,test_size=0.25,random_state=1)
# 模型实例化并加载数据---单个隐含层，100个神经元
net_0 = MLPClassifier()
net_0.fit(X_train,y_train)
print(net)
# print("训练集精度：",net.score(X_train,y_train))
# print("测试集精度：",net.score(X_test,y_test))

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)


In [166]:
print("训练集精度：",net_0.score(X_train,y_train))
print("测试集精度：",net_0.score(X_test,y_test))

训练集精度： 0.913145539906
测试集精度： 0.937062937063


In [169]:
# 标准化之后
std_x = StandardScaler()
data = std_x.fit_transform(breast_cancer.data)
print(data[0])
X_train,X_test,y_train,y_test = train_test_split(data,breast_cancer.target,test_size=0.25,random_state=0)
net = MLPClassifier()
net.fit(X_train,y_train)

[ 1.09706398 -2.07333501  1.26993369  0.9843749   1.56846633  3.28351467
  2.65287398  2.53247522  2.21751501  2.25574689  2.48973393 -0.56526506
  2.83303087  2.48757756 -0.21400165  1.31686157  0.72402616  0.66081994
  1.14875667  0.90708308  1.88668963 -1.35929347  2.30360062  2.00123749
  1.30768627  2.61666502  2.10952635  2.29607613  2.75062224  1.93701461]


D:\anaconda\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [170]:
print("训练集精度：",net.score(X_train,y_train))
print("测试集精度：",net.score(X_test,y_test))

训练集精度： 0.990610328638
测试集精度： 0.958041958042


### pytorch编写的神经网络框架

In [236]:
import torch
from torch import nn
from torch.autograd import Variable

In [237]:
# 标准化之前
X_train,X_test,y_train,y_test = train_test_split(breast_cancer.data,breast_cancer.target,test_size=0.25,random_state=1)
# 使用 Sequential 定义 3 层神经网络
net = nn.Sequential(
    nn.Linear(30, 100),
    nn.ReLU(),
    nn.Linear(100, 2)
)
# 定义 loss 函数---交叉熵
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), 1e-3) # 使用随机梯度下降，学习率 0.001

In [238]:
# 我们训练 10000 次
for e in range(10000):
    out = net(Variable(torch.from_numpy(X_train)).float())
    loss = criterion(out, Variable(torch.from_numpy(y_train)).long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if ((e + 1) % 1000 == 0) or e == 1:
        print('epoch: {}, loss: {}'.format(e+1, loss))

epoch: 2, loss: 308.1055908203125
epoch: 1000, loss: 0.19134202599525452
epoch: 2000, loss: 0.18094660341739655
epoch: 3000, loss: 0.1736592948436737
epoch: 4000, loss: 0.16670948266983032
epoch: 5000, loss: 0.16495977342128754
epoch: 6000, loss: 0.16151118278503418
epoch: 7000, loss: 0.15963903069496155
epoch: 8000, loss: 0.15406864881515503
epoch: 9000, loss: 0.15289217233657837
epoch: 10000, loss: 0.1520453244447708


In [240]:
out = net(Variable(torch.from_numpy(X_train)).float())
prediction = torch.max(out, 1)[1] # 1返回index  0返回原值
pred_y = prediction.data.numpy()
target_y = y_train
accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
print("训练集预测准确率",accuracy)
out = net(Variable(torch.from_numpy(X_test)).float())
prediction = torch.max(out, 1)[1] # 1返回index  0返回原值
pred_y = prediction.data.numpy()
target_y = y_test
accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
print("测试预测准确率",accuracy)

训练集预测准确率 0.9389671361502347
测试预测准确率 0.9300699300699301


In [241]:
# 标准化之后
std_x = StandardScaler()
data = std_x.fit_transform(breast_cancer.data)
X_train,X_test,y_train,y_test = train_test_split(data,breast_cancer.target,test_size=0.25,random_state=0)

In [242]:
# 使用 Sequential 定义 3 层神经网络
net = nn.Sequential(
    nn.Linear(30, 100),
    nn.ReLU(),
    nn.Linear(100, 2)
)
# 定义 loss 函数---交叉熵
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), 1e-3) # 使用随机梯度下降，学习率 0.001

In [243]:
for e in range(10000):
    out = net(Variable(torch.from_numpy(X_train)).float())
    loss = criterion(out, Variable(torch.from_numpy(y_train)).long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if ((e + 1) % 1000 == 0) or e == 1:
        print('epoch: {}, loss: {}'.format(e+1, loss))

epoch: 2, loss: 0.585678219795227
epoch: 1000, loss: 0.24396876990795135
epoch: 2000, loss: 0.16932208836078644
epoch: 3000, loss: 0.13616260886192322
epoch: 4000, loss: 0.11712930351495743
epoch: 5000, loss: 0.10467319935560226
epoch: 6000, loss: 0.09580877423286438
epoch: 7000, loss: 0.08913218230009079
epoch: 8000, loss: 0.08390714973211288
epoch: 9000, loss: 0.07968954741954803
epoch: 10000, loss: 0.07618918269872665


In [244]:
out = net(Variable(torch.from_numpy(X_train)).float())
prediction = torch.max(out, 1)[1] # 1返回index  0返回原值
pred_y = prediction.data.numpy()
target_y = y_train
accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
print("训练集预测准确率",accuracy)
out = net(Variable(torch.from_numpy(X_test)).float())
prediction = torch.max(out, 1)[1] # 1返回index  0返回原值
pred_y = prediction.data.numpy()
target_y = y_test
accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
print("测试预测准确率",accuracy)

训练集预测准确率 0.9835680751173709
测试预测准确率 0.965034965034965
